# Autoloading module

In [15]:
# auto loading
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config parser

In [40]:
import configparser
from os.path import dirname, join,abspath
config = configparser.ConfigParser()
config.read(join(dirname(abspath('')),'config.ini'))
username =config['FACEBOOK']['username']
username


'TanawatPoom'

# driver setup

In [46]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import sys
sys.path.append("..")

from sna import main

d = webdriver.Chrome(executable_path='../chromedriver.exe')
d.implicitly_wait(30)
d.maximize_window()

# Facebook

In [47]:
d.get("https://facebook.com/")

## login facebook

In [51]:
## set user
d.get("https://facebook.com/{}/friends".format(username))
friends = d.find_elements_by_class_name("_5qo4") # This could loop with scrolling

# Get more detail friend list

In [27]:
from sna.friend import Friend
from bs4 import BeautifulSoup 
import json
import time

def friend_info(driver,friend):
    friend_info =Friend()
    soup = BeautifulSoup(friend.get_attribute('innerHTML'), 'html.parser')
    divs = soup.find_all('div', {'class': 'fsl fwb fcb'})
    # get name/id
    for div in divs:
        friend_info.name=div.get_text()
        for a in div.find_all('a', href=True) :
            friend_info.profile_link=a['href'][:a['href'].find('?')]
            try:
                user_id=json.loads(a['data-gt'])['engagement']['eng_tid']
            except:
                user_id='xxxx'
        if user_id == 'xxxx':
            break
    friend_info.id =user_id
    divs = soup.find_all('a', {'class': '_39g5'})
    for div in divs:
        a=div.get_text().split()
        if len(a) ==3:
            friend_info.friend_mode =1
        else: 
            friend_info.friend_mode =2
        
        
        friend_info.n_friends= int(a[0].replace(',',''))
        friend_info.friend_link =div['href']
    return friend_info

In [63]:
# d.driver.get(mu_link)
def get_freind(username,driver):
    if '|' in username:
        fetch_url = "https://facebook.com/"+username.split('|')[1]+"/friends"
    else:
        fetch_url = "https://facebook.com/"+username+"/friends"
    driver.get(fetch_url)

    driver.get("https://facebook.com/{}/friends".format(username))
    l = 0
    last_l = -1
    friend_list = []
    while last_l != l:
        friends = d.find_elements_by_class_name("_5qo4")
        print('Interpreting Data')
        for friend in friends[l:]:
            friend_list.append(friend_info(driver,friend))
        print('Scrolling Down')
        for i in range(5):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
        last_l = l
        l = len(friends)
        print(len(friend_list), 'friends were collected.')
    return friend_list

In [64]:
get_freind(username,d)

Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.


In [62]:
for i in friend_list:
    print(i.name,i.n_friends,i.friend_mode,i.friend_link)

Chonlada Jungjaroensin 921 2 https://www.facebook.com/lookmaew.intania/friends
San Thongmeearkom 171 1 /browse/mutual_friends/?uid=1480622287
Thiradet Ngamwongwan 1194 2 https://www.facebook.com/daifukuzaadrive/friends
Kearthk Kittipatwong 741 2 https://www.facebook.com/KearthK/friends
Arin Pantisoontorn 2139 2 https://www.facebook.com/arin.pantisoontorn/friends
YoungPrach Enzeroohm 163 2 https://www.facebook.com/Enzeroohm/friends
Amornthep Kiatsakulphan 2267 2 https://www.facebook.com/amornthep.kiatsakulphan/friends
Thananya Phreeraphattanakarn 633 2 https://www.facebook.com/tphreera/friends
Poom Jeansathawong 109 1 /browse/mutual_friends/?uid=1057368017
Keng Prasit 216 1 /browse/mutual_friends/?uid=100001723472144
Wirasak Chomphu 31 1 /browse/mutual_friends/?uid=1651495145
Kritsada Limpawatkul 2086 2 https://www.facebook.com/kritsada.lpwk/friends
Pom Thittawee T 486 2 https://www.facebook.com/PomThittawee/friends
Ronnakrit Rattanasriampaipong 2379 2 https://www.facebook.com/rattanasr

In [54]:

# f=friends[0]
# soup = BeautifulSoup(f.get_attribute('innerHTML'), 'html.parser')
# divs = soup.find_all('div', {'class': 'fsl fwb fcb'})
# divs

[<div class="fsl fwb fcb"><a data-gt='{"engagement":{"eng_type":"1","eng_src":"2","eng_tid":"559506010","eng_data":[]},"coeff2_registry_key":"0406","coeff2_info":"AauIbPShDf_9Mt9OuIWBPgHdzvBLCWY3fUFLr1t4BHac_0h1OQ-N75hXrxFmeXguinU","coeff2_action":"1","coeff2_pv_signature":"1214754531"}' data-hovercard="/ajax/hovercard/user.php?id=559506010&amp;extragetparams=%7B%22hc_location%22%3A%22friends_tab%22%7D" data-hovercard-prefer-more-content-show="1" href="https://www.facebook.com/lookmaew.intania?fref=profile_friend_list&amp;hc_location=friends_tab">Chonlada Jungjaroensin</a></div>]

In [57]:
# #Mutual friends
# soup = BeautifulSoup(f.get_attribute('innerHTML'), 'html.parser')
# divs = soup.find_all('a', {'class': '_39g5'})
# print(divs[0].get_text())
# print(divs[0]['href'][:a['href'].find('?')])
# # mu_link= "https://facebook.com/"+divs[0]['href']
# # d.driver.get(mu_link)
# divs

In [47]:
len(friend_list)

123

In [6]:
from bs4 import BeautifulSoup 
import json
import time

def find_all_friends(username):
    if '|' in username:
        fetch_url = "https://facebook.com/"+username.split('|')[1]+"/friends"
    else:
        fetch_url = "https://facebook.com/"+username+"/friends"
    driver.get(fetch_url)
    l = 0
    last_l = -1
    friend_list = []
    while last_l != l:
        friends = driver.find_elements_by_class_name("_5qo4")
        print('Interpreting Data')
        for friend in friends[l:]:
            soup = BeautifulSoup(friend.get_attribute('innerHTML'), 'html.parser')
            divs = soup.find_all('div', {'class': 'fsl fwb fcb'})
            for div in divs:
                name=div.get_text()
                # print(name)
                for a in div.find_all('a', href=True) :
                    link=a['href'][:a['href'].find('?')]
                    try:
                        user_id=json.loads(a['data-gt'])['engagement']['eng_tid']
                    except:
                        user_id='xxxx'
                if user_id == 'xxxx':
                    break
                friend_list.append((name,link,user_id))
        print('Scrolling Down')
        for i in range(20):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
        last_l = l
        l = len(friends)
        print(len(friend_list), 'friends were collected.')
    return friend_list

In [ ]:
import random
import os.path

output_filename = 'result/output.csv'
def get_name(name,url,uid):
    if 'profile.php' in url:
        user_url = name+'|'+uid
    else:
        user_url = url[url.find('/',10)+1:]
    return user_url

if not os.path.exists(output_filename):
    source_user = username
else:
    file =open(output_filename,'r', encoding='utf-8')
    f_list = file.readlines()
    first_user_list = [line.split(',')[0] for line in f_list]
    source_user = random.choice(f_list).split(',')[1].strip()
    while source_user in first_user_list:
        source_user = random.choice(f_list).split(',')[1].strip()
    file.close()    

while True:
    friend_list = find_all_friends(source_user)
    new_friend_list = [(source_user,get_name(name,url,u_id)) for name,url,u_id in friend_list]
    file = open(output_filename,'a', encoding='utf-8')
    for c in new_friend_list:
        file.write(c[0]+','+c[1])
        file.write('\n')
    file.close()
    file = open(output_filename,'r', encoding='utf-8')
    f_list = file.readlines()
    first_user_list = [line.split(',')[0] for line in f_list]
    source_user = random.choice(f_list).split(',')[1].strip()
    while source_user in first_user_list:
        source_user = random.choice(f_list).split(',')[1].strip()
    file.close()

Interpreting Data
Scrolling Down
40 friends were collected.
Interpreting Data
Scrolling Down
320 friends were collected.
Interpreting Data
Scrolling Down
613 friends were collected.
Interpreting Data
Scrolling Down
803 friends were collected.
Interpreting Data
Scrolling Down
1043 friends were collected.
Interpreting Data
Scrolling Down
1258 friends were collected.
Interpreting Data
Scrolling Down
1308 friends were collected.
Interpreting Data
Scrolling Down
1308 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
80 friends were collected.
Interpreting Data
Scrolling Down
160 friends were collected.
Interpreting Data
Scrolling Down
160 friends were collected.
Interpreting Data
Scrolling Down
8 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling 

In [6]:
if not os.path.exists(output_filename):
    source_user = 'TanawatPoom'
else:
    file =open(output_filename,'r', encoding='utf-8')
    f_list = file.readlines()
    first_user_list = [line.split(',')[0] for line in f_list]
    source_user = random.choice(f_list).split(',')[1].strip()
    while source_user in first_user_list:
        source_user = random.choice(f_list).split(',')[1].strip()
    file.close()    

while True:
    friend_list = find_all_friends(source_user)
    new_friend_list = [(source_user,get_name(name,url,u_id)) for name,url,u_id in friend_list]
    file = open(output_filename,'a', encoding='utf-8')
    for c in new_friend_list:
        file.write(c[0]+','+c[1])
        file.write('\n')
    file.close()
    file = open(output_filename,'r', encoding='utf-8')
    f_list = file.readlines()
    first_user_list = [line.split(',')[0] for line in f_list]
    source_user = random.choice(f_list).split(',')[1].strip()
    while source_user in first_user_list:
        source_user = random.choice(f_list).split(',')[1].strip()
    file.close()

Interpreting Data
Scrolling Down
8 friends were collected.
Interpreting Data
Scrolling Down
27 friends were collected.
Interpreting Data
Scrolling Down
27 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
80 friends were collected.
Interpreting Data
Scrolling Down
240 friends were collected.
Interpreting Data
Scrolling Down
460 friends were collected.
Interpreting Data
Scrolling Down
618 friends were collected.
Interpreting Data
Scrolling Down
618 friends were collected.
Interpreting Data
Scrolling Down
618 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
60 friends were collected.
Interpreting Data
Scrolling Down
140 friends were collected.
Interpreting Data
Scrolling Down
220 friends were collected.
Interpreting Data
Scrolling Down
300 friends were collected.
Interpreting Data
Scrolling Down
300 friends were collected.
Interpreting Data
Scrolling Down

Scrolling Down
540 friends were collected.
Interpreting Data
Scrolling Down
640 friends were collected.
Interpreting Data
Scrolling Down
720 friends were collected.
Interpreting Data
Scrolling Down
820 friends were collected.
Interpreting Data
Scrolling Down
900 friends were collected.
Interpreting Data
Scrolling Down
1000 friends were collected.
Interpreting Data
Scrolling Down
1100 friends were collected.
Interpreting Data
Scrolling Down
1100 friends were collected.
Interpreting Data
Scrolling Down
20 friends were collected.
Interpreting Data
Scrolling Down
60 friends were collected.
Interpreting Data
Scrolling Down
140 friends were collected.
Interpreting Data
Scrolling Down
220 friends were collected.
Interpreting Data
Scrolling Down
300 friends were collected.
Interpreting Data
Scrolling Down
380 friends were collected.
Interpreting Data
Scrolling Down
460 friends were collected.
Interpreting Data
Scrolling Down
460 friends were collected.
Interpreting Data
Scrolling Down
20 frien

340 friends were collected.
Interpreting Data
Scrolling Down
420 friends were collected.
Interpreting Data
Scrolling Down
440 friends were collected.
Interpreting Data
Scrolling Down
440 friends were collected.
Interpreting Data
Scrolling Down
8 friends were collected.
Interpreting Data
Scrolling Down
11 friends were collected.
Interpreting Data
Scrolling Down
11 friends were collected.


TimeoutException: Message: timeout
  (Session info: chrome=76.0.3809.132)
